# 11. Local features - matching

In [1]:
import cv2

image_scene = cv2.imread("images/match_scene01_3.png", cv2.IMREAD_GRAYSCALE)
image_object_1 = cv2.imread("images/match_scene01_1.png", cv2.IMREAD_GRAYSCALE)
image_object_2 = cv2.imread("images/match_scene01_2.png", cv2.IMREAD_GRAYSCALE)

Detect the keypoints in two different images (ex: with different scales, taken from different viewpoints, …) and draw
the found matches, like in figure 3. Try different sets of images, keypoint detectors, matchers and visualization of the
matches:
 SIFT or ORB for keypoint detection;
 Brute Force or FLANN for matching, with the alternatives match() or KnnMatch(); choose an appropriate
distance measure;
 drawMatches() (match()) or drawMatchesKnn() (KnnMatch()) for displaying the results.

In [4]:
# Brute-Force matching with orb descriptors

orb = cv2.ORB_create()

kp0, des0 = orb.detectAndCompute(image_scene, None)
kp1, des1 = orb.detectAndCompute(image_object_1, None)
kp2, des2 = orb.detectAndCompute(image_object_2, None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches1 = bf.match(des1, des0)
matches2 = bf.match(des2, des0)

matches1 = sorted(matches1, key=lambda x: x.distance)
matches2 = sorted(matches2, key=lambda x: x.distance)

res1 = cv2.drawMatches(
    image_object_1,
    kp1,
    image_scene,
    kp0,
    matches1[:10],
    None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
)
res2 = cv2.drawMatches(
    image_object_2,
    kp2,
    image_scene,
    kp0,
    matches2[:10],
    None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
)

cv2.imshow("Object 1", res1)
cv2.imshow("Object 2", res2)

cv2.waitKey(0)
cv2.destroyAllWindows()


In [10]:
# Brute-Force Matching with SIFT Descriptors and Ratio Test

sift = cv2.SIFT_create()

kp0, des0 = sift.detectAndCompute(image_scene, None)
kp1, des1 = sift.detectAndCompute(image_object_1, None)
kp2, des2 = sift.detectAndCompute(image_object_2, None)

bf = cv2.BFMatcher()
matches1 = bf.knnMatch(des1, des0, k=2)
matches2 = bf.knnMatch(des2, des0, k=2)

good1 = []
for m, n in matches1:
    if m.distance < 0.75 * n.distance:
        good1.append([m])

good2 = []
for m, n in matches2:
    if m.distance < 0.75 * n.distance:
        good2.append([m])

res1 = cv2.drawMatchesKnn(
    image_object_1,
    kp1,
    image_scene,
    kp0,
    good1,
    None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
)
res2 = cv2.drawMatchesKnn(
    image_object_2,
    kp2,
    image_scene,
    kp0,
    good2,
    None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
)

cv2.imshow("Object 1", res1)
cv2.imshow("Object 2", res2)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)

sift = cv2.SIFT_create()

kp0, des0 = sift.detectAndCompute(image_scene, None)
kp1, des1 = sift.detectAndCompute(image_object_1, None)
kp2, des2 = sift.detectAndCompute(image_object_2, None)

flann = cv2.FlannBasedMatcher(index_params, search_params)

matches1 = flann.knnMatch(des1, des0, k=2)
matches2 = flann.knnMatch(des2, des0, k=2)

matchesMask1 = [[0, 0] for _ in range(len(matches1))]
matchesMask2 = [[0, 0] for _ in range(len(matches2))]

for i, (m, n) in enumerate(matches1):
    if m.distance < 0.7 * n.distance:
        matchesMask1[i] = [1, 0]

for i, (m, n) in enumerate(matches2):
    if m.distance < 0.7 * n.distance:
        matchesMask2[i] = [1, 0]

draw_params1 = dict(
    matchColor=(0, 255, 0),
    singlePointColor=(255, 0, 0),
    matchesMask=matchesMask1,
    flags=cv2.DrawMatchesFlags_DEFAULT,
)

draw_params2 = dict(
    matchColor=(0, 255, 0),
    singlePointColor=(255, 0, 0),
    matchesMask=matchesMask2,
    flags=cv2.DrawMatchesFlags_DEFAULT,
)

res1 = cv2.drawMatchesKnn(
    image_object_1, kp1, image_scene, kp0, matches1, None, **draw_params1
)
res2 = cv2.drawMatchesKnn(
    image_object_2, kp2, image_scene, kp0, matches2, None, **draw_params2
)

cv2.imshow("Object 1", res1)
cv2.imshow("Object 2", res2)

cv2.waitKey(0)
cv2.destroyAllWindows()
